In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../Datasets/iris.csv', sep=';', names=["sepal length", "sepal width", "petal length", "petal width", "class"])

In [3]:
df.head()

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [4]:
def gini(classes):
    groups = {}
    for v in classes:
        if not v in groups:
            groups[v] = 0
        groups[v] = groups[v] + 1 
    #print("groups", groups)
    return 1 - sum(map(lambda p: (p / len(classes)) ** 2, groups.values()))

def weighted_gini(classes, split_index):
    g1 = gini(classes[:split_index])
    g2 = gini(classes[split_index:])
    #print("g1", g1, "g2", g2)
    g1w = g1 * (split_index / len(classes))
    g2w = g2 * ((len(classes) - split_index) / len(classes))
    #print("g1w", g1w, "g2w", g2w)
    return g1w + g2w

def find_best_gini(df):
    i = -1 # column
    g = 100 # worst gini
    s = -1 # split
    v = None # split value
    
    for column_index, column in enumerate(df.columns[:-1]):
        #print("COLUMN", column)
        df = df.sort_values(by=column)
        #display(df.head())
        for split in range(1, len(df)):
            #print(split)
            v1 = df.iloc[split - 1, column_index]
            #print("v1", v1)
            v2 = df.iloc[split, column_index]
            #print("v2", v2)
            if v1 == v2:
                # edge case when there are only records with same value 
                #print("Equal indexes {}".format(v1))
                continue
            newG = weighted_gini(df[df.columns[-1]], split)
            if newG < g:
                i, g, s, v = column, newG, split, (v1 + v2) / 2
                #print("column=", i, "g=", g, "s=", s, "v=", v, v1, v2)
        #print("column=", i, "g=", g, "s=", s, "v=", v)
    #print("column=", i, "g=", g, "s=", s, "v=", v)
    return i, g, v

In [5]:
class Node:
    def __init__(self, column, gini, value, samples):
        self.column = column
        self.gini = gini
        self.value = value
        self.samples = samples
        self.left = None
        self.right = None
    
    def __repr__(self):
        return "column={}, gini={}, value={}, samples={}".format(self.column, self.gini, self.value, self.samples)
    
    def is_leaf(self):
        return self.left is None and self.right is None

class DecisionTree:
    def __init__(self, root):
        self.root = root
    
    def predict(self, values):
        node = self.root
        while not node.is_leaf():
            if values[node.column] <= node.value:
                #print("go left")
                node = node.left
            else:
                #print("go right")
                node = node.right
        return node.value

def build_decision_tree(df):
    def build_node(df):
        column, gini, value = find_best_gini(df)
        #display(df.head())
        node = Node(column, gini, value, len(df))
        display(node)
        return node
    
    def build_node_recursively(parent, df, depth):
        print("depth", depth, parent, df.columns)
        if len(df) == 0:
            print("fail")
            return
        values = df[df.columns[-1]].unique()
        #print(values)
        if len(values) == 1:
            print("leaf", values[0])
            parent.value = values[0]
            return
        
        left_df = df[df[parent.column] <= parent.value]
        parent.left = build_node(left_df)
        print("build left")
        build_node_recursively(parent.left, left_df, depth + 1)
        
        right_df = df[df[parent.column] > parent.value]
        parent.right = build_node(right_df)
        print("build right")
        build_node_recursively(parent.right, right_df, depth + 1)
        
    root = build_node(df)
    build_node_recursively(root, df, 0)
    
    return DecisionTree(root)

tree = build_decision_tree(df)

column=petal length, gini=0.3333333333333333, value=2.45, samples=150

depth 0 column=petal length, gini=0.3333333333333333, value=2.45, samples=150 Index(['sepal length', 'sepal width', 'petal length', 'petal width', 'class'], dtype='object')


column=sepal length, gini=0.0, value=4.35, samples=50

build left
depth 1 column=sepal length, gini=0.0, value=4.35, samples=50 Index(['sepal length', 'sepal width', 'petal length', 'petal width', 'class'], dtype='object')
leaf 0


column=petal width, gini=0.11030595813204513, value=1.75, samples=100

...

build left
depth 4 column=-1, gini=100, value=None, samples=1 Index(['sepal length', 'sepal width', 'petal length', 'petal width', 'class'], dtype='object')
leaf 1


column=sepal length, gini=0.0, value=6.1, samples=2

build right
depth 4 column=sepal length, gini=0.0, value=6.1, samples=2 Index(['sepal length', 'sepal width', 'petal length', 'petal width', 'class'], dtype='object')
leaf 2


column=sepal length, gini=0.0, value=5.65, samples=43

build right
depth 3 column=sepal length, gini=0.0, value=5.65, samples=43 Index(['sepal length', 'sepal width', 'petal length', 'petal width', 'class'], dtype='object')
leaf 2


In [6]:
correct = 0
for i in range(len(df)):
    label = df.iloc[i]['class']
    predicted = tree.predict(df.iloc[i])
    if label == predicted:
        correct += 1
    
print("Correct={}/{}".format(correct, len(df)))

Correct=150/150


In [7]:
def print_tree(current_node, indent='', last='updown'):
    def name(node):
        if current_node.is_leaf():
            return "class={}, s={}".format(node.value, node.samples)
        else:
            return "{}, v={}, s={}".format(node.column, node.value, node.samples)

    children = lambda node: [] if node.is_leaf() else [node.left, node.right]
    nb_children = lambda node: sum(nb_children(child) for child in children(node)) + 1
    size_branch = {child: nb_children(child) for child in children(current_node)}

    """ Creation of balanced lists for "up" branch and "down" branch. """
    up = sorted(children(current_node), key=lambda node: nb_children(node))
    down = []
    while up and sum(size_branch[node] for node in down) < sum(size_branch[node] for node in up):
        down.append(up.pop())

    """ Printing of "up" branch. """
    for child in up:
        next_last = 'up' if up.index(child) is 0 else ''
        next_indent = '{0}{1}{2}'.format(indent, ' ' if 'up' in last else '│', ' ' * len(name(current_node)))
        print_tree(child, next_indent, next_last)

    """ Printing of current node. """
    if last == 'up': start_shape = '┌'
    elif last == 'down': start_shape = '└'
    elif last == 'updown': start_shape = ' '
    else: start_shape = '├'

    if up: end_shape = '┤'
    elif down: end_shape = '┐'
    else: end_shape = ''

    print('{0}{1}{2}{3}'.format(indent, start_shape, name(current_node), end_shape))

    """ Printing of "down" branch. """
    for child in down:
        next_last = 'down' if down.index(child) is len(down) - 1 else ''
        next_indent = '{0}{1}{2}'.format(indent, ' ' if 'down' in last else '│', ' ' * len(name(current_node)))
        print_tree(child, next_indent, next_last)

display(["Iris-setosa", "Iris-versicolor", "Iris-virginica"])
print_tree(tree.root)

['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']

                            ┌class=0, s=50
 petal length, v=2.45, s=150┤
                            │                                                     ┌class=2, s=43
                            │                          ┌petal length, v=4.85, s=46┤
                            │                          │                          │                         ┌class=1, s=1
                            │                          │                          └sepal length, v=5.95, s=3┤
                            │                          │                                                    └class=2, s=2
                            └petal width, v=1.75, s=100┤
                                                       │                                                    ┌class=1, s=47
                                                       │                          ┌petal width, v=1.65, s=48┤
                                                       │                          │                   

In [8]:
def print_tree_simple(node, depth):
    if node == None:
        return
    
    type = "class" if node.is_leaf() else "value"
    print("{}column={}, {}={}".format(' ' * depth * 5, node.column, type, node.value))
    print_tree_simple(node.left, depth + 1)
    print_tree_simple(node.right, depth + 1)

print_tree_simple(tree.root, 0)

column=petal length, value=2.45
     column=sepal length, class=0
     column=petal width, value=1.75
          column=petal length, value=4.95
               column=petal width, value=1.65
                    column=sepal length, class=1
                    column=-1, class=2
               column=petal width, value=1.55
                    column=sepal length, class=2
                    column=sepal length, value=6.95
                         column=sepal length, class=1
                         column=-1, class=2
          column=petal length, value=4.85
               column=sepal length, value=5.95
                    column=-1, class=1
                    column=sepal length, class=2
               column=sepal length, class=2
